# AI Anomaly Detection Workflow

This notebook walks through the entire AI setup—loading baseline data, training IsolationForest models, saving them, loading new (unlabeled) data, predicting anomalies feature‐by‐feature, and saving a labeled CSV. You can run each cell in order.

---

In [ ]:
### Cell 1 – Import Libraries and Define Paths
import os
from pathlib import Path
import pandas as pd
import pickle
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

# Define project‐root and subfolders
project_root = Path().resolve()
data_dir     = project_root / "data"
model_dir    = project_root / "model"
results_dir  = data_dir / "results"

# Ensure folders exist
data_dir.mkdir(exist_ok=True)
model_dir.mkdir(exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

# File paths
baseline_path      = data_dir / "baseline.csv"
new_data_path      = data_dir / "new_data.csv"
model_path         = model_dir / "anomaly_models.pkl"
labeled_output_path = results_dir / "new_data_labeled.csv"